I need to convert a list of 32 bits in a list of 24 bits

### The program as it is works so don't touch anything

In [ ]:
import numpy as np
import struct
from random import seed, random
from math import sin, pi
from matplotlib import pyplot as plt

In [ ]:
pack_format = 'l' # long int, 4 bytes
length = 500000
precision = 32
amplitude = (2**precision)/2-1
frequency = 440
rate = 192000

In [ ]:
# Generate a sin()

values = [amplitude*sin(2*pi*frequency*i/rate) for i in range(length)]
values = np.array(values).astype(np.int32)

In [ ]:
# Convert values into bytes

packed_data = struct.pack('<' + str(length) + pack_format, *values)
print(packed_data)
print(len(packed_data))
packed_data[::4]

In [ ]:
first_bytes = [packed_data[i:i+4:4] for i in range(0, len(packed_data), 4)]
print(first_bytes)

listed = [packed_data[i:i+4] for i in range(0, len(packed_data), 4)]
print(listed)

print(len(first_bytes))

In [ ]:
def replace_byte_in_place(byte_string, index, new_byte):
  """
  This function replaces a byte at a specific index in a byte string (modifies in-place).

  Args:
      byte_string: The original byte string.
      index: The index of the byte to replace (0-based indexing).
      new_byte: The byte value (integer between 0 and 255) to replace the byte at the index.

  Raises:
      TypeError: If the input is not a byte string, the index or new_byte is not an integer.
      ValueError: If the index is outside the valid range of the byte string or 
                   the new_byte value is outside the valid range (0-255).
  """
  if not isinstance(byte_string, bytes):
    raise TypeError("Input must be a byte string")
  if not isinstance(index, int) or not isinstance(new_byte, int):
    raise TypeError("Index and new_byte must be integers")
  if not (0 <= index < len(byte_string)):
    raise ValueError("Index is outside the byte string range")
  if not (0 <= new_byte <= 255):
    raise ValueError("New byte value must be between 0 and 255")
  byte_array = bytearray(byte_string)
  byte_array[index] = new_byte
  return bytes(byte_array)

def remove_byte(byte_string, index):
  """
  This function removes a byte at a specific index in a byte string and returns a new byte string.

  Args:
      byte_string: The original byte string.
      index: The index of the byte to remove (0-based indexing).

  Raises:
      TypeError: If the input is not a byte string or the index is not an integer.
      ValueError: If the index is outside the valid range of the byte string.
  """
  if not isinstance(byte_string, bytes):
    raise TypeError("Input must be a byte string")
  if not isinstance(index, int):
    raise TypeError("Index must be an integer")
  if not (0 <= index < len(byte_string)):
    raise ValueError("Index is outside the byte string range")
  return byte_string[:index] + byte_string[index + 1:]

def insert_byte(byte_string, index, new_byte):
  """
  This function inserts a byte at a specific index in a byte string (modifies in-place).

  Args:
      byte_string: The original byte string.
      index: The index to insert the byte at (0-based indexing).
      new_byte: The byte value (integer between 0 and 255) to insert.

  Raises:
      TypeError: If the input is not a byte string, the index or new_byte is not an integer.
      ValueError: If the index is outside the valid range of the byte string or 
                   the new_byte value is outside the valid range (0-255).
  """
  if not isinstance(byte_string, bytes):
    raise TypeError("Input must be a byte string")
  if not isinstance(index, int) or not isinstance(new_byte, int):
    raise TypeError("Index and new_byte must be integers")
  if not (0 <= index <= len(byte_string)):
    raise ValueError("Index is outside the byte string range")
  if not (0 <= new_byte <= 255):
    raise ValueError("New byte value must be between 0 and 255")
  byte_array = bytearray(byte_string)
  byte_array.insert(index, new_byte)
  return bytes(byte_array)

In [ ]:
write_bytes = []

for idx in range(len(first_bytes)):
    removed_bytes = remove_byte(listed[idx], 0)
    write_bytes.append(removed_bytes)

print(write_bytes)

In [ ]:
new_bytes = []

for idx in range(len(first_bytes)):
    #if idx == 743:
        #print('first byte:', first_bytes[idx])
        #print('original byte:', listed[idx])
        #listed[idx] = replace_byte_in_place(listed[idx], 0, 0x00)
        #print('new byte:', listed[idx])

    #new_bytes.append(listed[idx].replace(first_bytes[idx], b'\x00'))
    #new_bytes.append(replace_byte_in_place(listed[idx], 0, 0x00))
    new_bytes.append(insert_byte(write_bytes[idx], 0, 0x00))

print(new_bytes)

In [ ]:
bytes_arr = bytearray()

for ii in range(len(new_bytes)):
    bytes_arr.extend(new_bytes[ii])
# convert byte_arr to bytes_string via

bytes_list = bytes(bytes_arr)

In [ ]:
unpacked_data = struct.unpack('<' + str(length) + pack_format, bytes_list)
print(unpacked_data)

In [ ]:
print(len(unpacked_data))
print(len(values))

In [ ]:
x = []
y = []

for jj in range(len(unpacked_data)):
    difference = unpacked_data[jj] - values[jj]
    if False:
        print('index: %i, unpacked value: %i, original value: %i, difference: %i' % (jj, unpacked_data[jj], values[jj], difference))
        x.append(unpacked_data[jj])
        y.append(values[jj])

In [ ]:
plt.plot(values, 'o', markersize = 1)
plt.plot(unpacked_data, 'o', markersize = 1)
plt.xlim(0,1000)

In [ ]:
plt.plot(unpacked_data-values, 'o', markersize = 1)
plt.xlim(0,10000)

nook = []
for ii in range(len(unpacked_data)):
    if unpacked_data[ii]-values[ii] > 1000:
        nook.append(unpacked_data[ii]-values[ii])

print(len(nook))
print(len(nook)/len(unpacked_data))